In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
from torch.utils.tensorboard import SummaryWriter

import numpy as np
import pandas as pd
import hashlib
import shutil
import glob
import time
import re
import os

from tqdm import tqdm
from datetime import datetime
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score
    

In [8]:
class Net(nn.Module):
    def __init__(self, sequenceSize=20000, embeddingDim=128, vocabularySize=2**16, filterWidth=5, filterNumber=1024):
        super(Net, self).__init__()
        self.sequenceSize   = sequenceSize
        self.embeddingDim   = embeddingDim
        self.vocabularySize = vocabularySize
        self.filterWidth    = filterWidth
        self.filterNumber   = filterNumber 
        
        self.embedding = nn.Embedding(self.vocabularySize, self.embeddingDim)
        self.conv = nn.Sequential(
                            nn.Conv2d(1, self.filterNumber, (self.filterWidth, self.embeddingDim)),
                            nn.BatchNorm2d(self.filterNumber),
                            nn.ReLU()
                        )
        
        self.fc = nn.Sequential(
                        nn.Linear(self.filterNumber , 512),
                        nn.BatchNorm1d(512),
                        nn.ReLU(),
            
                        nn.Linear(512, 256),
                        nn.BatchNorm1d(256),
                        nn.ReLU(),
                        
                        nn.Linear(256, 1),
                        nn.Sigmoid()
                    )

    def forward(self, x):
        x = self.embedding(x)
        #print(x.size())
        
        x = self.conv(x)
        #print(x.size())
        
        x = x.max(dim=2)[0]
        #print(x.size())

        x = x.view(-1,  self.filterNumber)
        x = self.fc(x)
        return x

class SampleDataset(Dataset):
    def __init__(self, filePathList, labels, sequenceSize=20000, featureName='functionMethodCallsArgs'):
        self.filePathList = filePathList
        self.labels = labels
        self.sequenceSize = sequenceSize
        self.featureName = featureName
        
    def __len__(self):
        return len(self.filePathList)

    def __getitem__(self, idx):
        df = pd.read_parquet(self.filePathList[idx])
        seed = int(round(time.time()%1, 6) * 1000000)
        x = np.concatenate(df.iloc[np.random.RandomState(seed).permutation(len(df))][self.featureName].values)

        if len(x) > self.sequenceSize:
            x = x[:self.sequenceSize]
        else:
            x = np.concatenate((x, np.zeros([self.sequenceSize - len(x)])))
            
        sample = torch.from_numpy(x)
        return (sample.long(), self.labels[idx], self.filePathList[idx])

def train(model, optimizer, dataLoader, device):
    running_loss  = 0.0  
    label_lst     = list()
    predicted_lst = list()

    model.train()
    for inputs, labels, _ in dataLoader:
        
        #
        inputs = inputs.unsqueeze(1).to(device)
        labels = labels.to(device)

        #
        optimizer.zero_grad()

        #
        outputs = model(inputs)
        predicted = (outputs > 0.5).squeeze().long()
        loss = F.binary_cross_entropy(outputs.squeeze(), labels.float())

        #
        loss.backward()
        optimizer.step()

        #
        label_lst.append(labels.cpu().numpy())
        predicted_lst.append(predicted.cpu().numpy())        
        running_loss += loss.item() 

    labels    = np.concatenate(label_lst)
    predicted = np.concatenate(predicted_lst)
    loss      = running_loss / len(predicted)
    
    return labels, predicted, loss

def assess(model, dataLoader, device):
    running_loss  = 0.0  
    label_lst     = list()
    predicted_lst = list()
    proba_lst     = list()
    path_lst      = list()

    with torch.no_grad():
        model.eval()
        for inputs, labels, paths in dataLoader:
            #
            inputs = inputs.unsqueeze(1).to(device)
            labels = labels.to(device)

            #
            outputs = model(inputs)
            predicted = (outputs > 0.5).squeeze().long()
            loss = F.binary_cross_entropy(outputs.squeeze(), labels.float())

            #
            if len(inputs) > 1:
                label_lst.append(labels.cpu().numpy())
                predicted_lst.append(predicted.cpu().numpy())
                proba_lst.append(outputs.squeeze().cpu().numpy())
                path_lst.append(paths)
                running_loss += loss.item() 
    
    labels    = np.concatenate(label_lst)
    predicted = np.concatenate(predicted_lst)
    proba     = np.concatenate(proba_lst)
    paths     = np.concatenate(path_lst)
    loss      = running_loss / len(predicted)
    
    return labels, predicted, loss, proba, paths

def trainModel(ws, modelTag, epochNum, trainLoader, validLoader, device, lr=3e-4, weightDecay=9e-5):
    #
    model  = Net()
    model  = model.to(device)
    optimizer = optim.Adam(model.parameters(), lr=lr, weight_decay=weightDecay)
    scheduler = ReduceLROnPlateau(optimizer, 'min', verbose=True, patience=5, factor=0.8)

    outputlogFilePath = f'./traces/{ws}/logs'
    outputtracesPath  = f'./traces/{ws}'
    #shutil.rmtree(outputtracesPath)
    #os.mkdir(outputtracesPath)

    result_lst = list()

    message = '----------'
    with open(outputlogFilePath, 'a') as writer:
        writer.write(message + '\n')
    print(message)
    
    for epoch in range(epochNum):

        tlabel, tpredicted, tloss = train(model, optimizer, trainLoader, device)
        vlabel, vpredicted, vloss, vproba, vproba = assess(model, validLoader, device)

        message  = f'Train: {modelTag} '
        message += '[{:04d}] '.format(epoch)

        tf1score  = f1_score(tlabel, tpredicted)
        message  += 'TF1: {:2.4f}, '.format(tf1score*100)
        message  += 'Tloss: {:2.8f}, '.format(tloss)

        vf1score  = f1_score(vlabel, vpredicted)
        message  += 'VF1: {:2.4f}, '.format(vf1score*100)
        message  += 'VLoss: {:2.8f},'.format(vloss)  
    
        with open(outputlogFilePath, 'a') as writer:
            writer.write(message + '\n')
        print(message)

        modelOutputPath = f'{outputtracesPath}/model_{modelTag}_{epoch:03d}.pth'
        torch.save(model.state_dict(), modelOutputPath)
        result_lst.append((epoch, modelOutputPath, vlabel, vpredicted, vproba, vf1score, vloss, tf1score, tloss))

        scheduler.step(tloss)

    df = pd.DataFrame(result_lst, 
                      columns=['epoch', 'path', 'labels', 'predicted', 'proba', 'vf1score', 'vloss', 'tf1score', 'tloss'])
    df.to_parquet(f'{outputtracesPath}/{modelTag}.parquet')

    message = '----------'
    with open(outputlogFilePath, 'a') as writer:
        writer.write(message + '\n')
    print(message)

    return df

def evaluate(ws, modelPathList, dataloader, device, numberFragments=1):
    modelResultList = []
    outputlogFilePath = f'./traces/{ws}/logs'
    
    for modelPath in modelPathList:
        for fragment in range(numberFragments):
            mdl = Net().to(device)
            mdl.load_state_dict(torch.load(modelPath))
            mdl.eval()
            modelResult = assess(mdl, dataloader, device)
            modelF1Score = f1_score(modelResult[0], modelResult[1])
            modelResultList.append((modelPath, modelF1Score,) + modelResult)
            message  = f'Evaluate: '
            message += f'ModelPath={modelPath} Fragment={fragment:02d} '
            message += f'score={modelF1Score}'
            print(message)
            with open(outputlogFilePath, 'a') as writer:
                writer.write(message + '\n')
    return pd.DataFrame(modelResultList, columns=['name', 'f1score', 'Truth', 'Predicted', 'loss', 'Proba', 'Path'])

def getDataloaders(dataset_df, batchSize=32, numWorkers=16, trainPercentage=0.7, validPercentage=0.8):
    rand_idx = np.random.permutation(len(dataset_df))
    train_df = dataset_df.iloc[rand_idx[:int(trainPercentage * len(dataset_df))]]
    valid_df = dataset_df.iloc[rand_idx[int(trainPercentage * len(dataset_df)):int(validPercentage * len(dataset_df))]]
    test_df  = dataset_df.iloc[rand_idx[int(validPercentage * len(dataset_df)):]]

    print(len(train_df))
    print(train_df.label.value_counts())
    print(len(valid_df))
    print(valid_df.label.value_counts())
    print(len(test_df))
    print(test_df.label.value_counts())
    
    trainDataset = SampleDataset(train_df.filePath.values, train_df.label.values)
    trainLoader  = DataLoader(trainDataset, batch_size=batchSize, shuffle=True, num_workers=numWorkers)

    validDataset = SampleDataset(valid_df.filePath.values, valid_df.label.values)
    validLoader  = DataLoader(validDataset, batch_size=2*batchSize, shuffle=False, num_workers=numWorkers)

    testDataset = SampleDataset(test_df.filePath.values, test_df.label.values)
    testLoader  = DataLoader(testDataset,  batch_size=2*batchSize, shuffle=False, num_workers=numWorkers)
    
    return trainLoader, validLoader, testLoader

def evalDataset(ws, result_df, probaUpperBorn = 0.9,  probaLowerBorn = 0.1):
    outputlogFilePath = f'./traces/{ws}/logs'
    results   = np.vstack(result_df.Proba.values)

    truth       = result_df.Truth.iloc[0]
    paths       = result_df.Path.iloc[0]
    result_mean = results.mean(axis=0)
    predicted   = (result_mean > 0.5).astype('int')
    f1score     = f1_score(truth, predicted)

    vtruth        = truth[(result_mean >= probaUpperBorn) | (result_mean <= probaLowerBorn)]
    vpaths        = paths[(result_mean >= probaUpperBorn) | (result_mean <= probaLowerBorn)]
    vresult_prob  = result_mean[(result_mean >= probaUpperBorn) | (result_mean <= probaLowerBorn)]
    vpredicted    = (vresult_prob > 0.5).astype('int')
    vcoverage     = (len(vtruth)/len(truth))
    vextendSize   = len(vtruth)
    vf1score      = f1_score(vtruth, vpredicted)

    etruth       = truth[(result_mean < probaUpperBorn) & (result_mean > probaLowerBorn)]
    epaths       = paths[(result_mean < probaUpperBorn) & (result_mean > probaLowerBorn)]
    eresult_prob = result_mean[(result_mean < probaUpperBorn) & (result_mean > probaLowerBorn)]
    epredicted    = (eresult_prob > 0.5).astype('int')
    ecoverage     = (len(etruth)/len(truth))
    erestSize     = len(etruth)
    ef1score      = f1_score(etruth, epredicted)

    message  = f'Extend: '
    message += f'f1score={f1score*100:2.4f}, '
    message += f'vcoverage={vcoverage*100:2.4f}, vf1score={vf1score*100:2.4f}, vexentdSize={vextendSize}, '
    message += f'ecoverage={ecoverage*100:2.4f}, ef1score={ef1score*100:2.4f}, erestSize={erestSize}'

    print(message)
    with open(outputlogFilePath, 'a') as writer:
        writer.write(message + '\n')

In [2]:
# 
ws               = 'detectionWS01'
epochNum         = 100
device           = torch.device('cuda:0')
ensembleSize     = 20
trainPercentageParam = 0.7
validPercentageParam = 0.8

outputlogFilePath = f'./traces/{ws}/logs'
outputtracesPath  = f'./traces/{ws}'
os.mkdir(outputtracesPath)

In [71]:
def evalDataset(result_df, probaUpperBorn = 0.9,  probaLowerBorn = 0.1):
    results   = np.vstack(result_df.Proba.values)

    truth         = result_df.Truth.iloc[0]
    paths         = result_df.Path.iloc[0]
    result_mean   = results.mean(axis=0)
    predicted     = (result_mean > 0.5).astype('int')
    f1score       = f1_score(truth, predicted)
    recall        = recall_score(truth, predicted)
    precision     = precision_score(truth, predicted)
    
    vtruth        = truth[(result_mean >= probaUpperBorn) | (result_mean <= probaLowerBorn)]
    vpaths        = paths[(result_mean >= probaUpperBorn) | (result_mean <= probaLowerBorn)]
    vresult_prob  = result_mean[(result_mean >= probaUpperBorn) | (result_mean <= probaLowerBorn)]
    vpredicted    = (vresult_prob > 0.5).astype('int')
    vcoverage     = (len(vtruth)/len(truth))
    vextendSize   = len(vtruth)
    vf1score      = f1_score(vtruth, vpredicted)
    vrecall       = recall_score(vtruth, vpredicted)
    vprecision    = precision_score(vtruth, vpredicted)
    
    message  = f'Extend: '
    message += f'f1score={f1score*100:2.4f}, recall={recall*100:2.4f}, precision={precision*100:2.4f}, '
    message += f'vf1score={vf1score*100:2.4f}, vrecall={vrecall*100:2.4f}, vprecision={vprecision*100:2.4f}, vcoverage={vcoverage*100:2.4f}, '
    #message += f'ecoverage={ecoverage*100:2.4f}, ef1score={ef1score*100:2.4f}, erestSize={erestSize}'

    print(message)

In [91]:
dataset_metaList = glob.glob('traces/detectionWS01/*.pickle')

In [95]:
dataset_metaList = ['traces/detectionWS01/genome.pickle',
 'traces/detectionWS01/drebin.pickle',
 'traces/detectionWS01/maldozer.pickle',
 'traces/detectionWS01/amd.pickle',
'traces/detectionWS01/vshare.pickle',]

In [96]:
dbPath = dataset_metaList[0]
ensembleSize = 20

In [100]:
for ensembleSize in range(1, 21):
    print(ensembleSize)
    for dbPath in dataset_metaList:
        #print(dbPath)
        df = pd.read_pickle(dbPath)

        models_df = df.models.iloc[0]
        result_df = df.evalResuls.iloc[0]
        models_df.sort_values(by=['vloss', 'tloss'], inplace=True)
        selectedModelPaths = models_df.path.iloc[:ensembleSize].tolist()
        current_result_df = result_df.loc[result_df.name.isin(selectedModelPaths)]

        evalDataset(current_result_df)

1
Extend: f1score=99.5633, recall=99.1304, precision=100.0000, vf1score=99.5536, vrecall=99.1111, vprecision=100.0000, vcoverage=99.6032, 
Extend: f1score=99.0637, recall=99.3427, precision=98.7862, vf1score=99.4334, vrecall=99.5274, vprecision=99.3396, vcoverage=99.3787, 
Extend: f1score=98.0832, recall=98.0338, precision=98.1327, vf1score=99.0309, vrecall=98.8903, vprecision=99.1718, vcoverage=98.0772, 
Extend: f1score=99.3836, recall=99.3728, precision=99.3943, vf1score=99.5852, vrecall=99.5201, vprecision=99.6505, vcoverage=99.4340, 
Extend: f1score=95.0341, recall=95.5785, precision=94.4959, vf1score=98.1545, vrecall=97.9789, vprecision=98.3306, vcoverage=92.0588, 
2
Extend: f1score=99.3464, recall=99.1304, precision=99.5633, vf1score=99.5536, vrecall=99.1111, vprecision=100.0000, vcoverage=99.4709, 
Extend: f1score=99.3433, recall=99.4366, precision=99.2502, vf1score=99.6198, vrecall=99.6198, vprecision=99.6198, vcoverage=98.9438, 
Extend: f1score=98.3702, recall=98.1346, precisi

In [105]:
meta_df = pd.read_parquet('/ws/mnt/local/data/output/obfus/praguard_meta.parquet')

In [107]:
meta_df.label.value_counts()

0    19690
1     9845
Name: label, dtype: int64

In [102]:
df = pd.read_pickle(dbPath)

In [103]:
df

,TimeTag,models,evalResuls
0,praguard,epoch ...,...


In [137]:
dataset_metaList = glob.glob('traces/obfusWS01/*.pickle')
#dbPath = 'traces/obfusWS01/'

for dbPath in dataset_metaList:
    print(dbPath)
    for ensembleSize in range(1, 12):
        #print(ensembleSize)

        df = pd.read_pickle(dbPath)

        models_df = df.models.iloc[0]
        result_df = df.evalResuls.iloc[0]
        
        models_df.sort_values(by=[ 'vloss', 'tloss'], ascending=True, inplace=True)
        selectedModelPaths = models_df.path.iloc[:ensembleSize].tolist()
        current_result_df = result_df.loc[result_df.name.isin(selectedModelPaths)]

        evalDataset(current_result_df)

traces/obfusWS01/nontrivialjunk.pickle
Extend: f1score=99.4773, recall=99.4088, precision=99.5459, vf1score=99.7621, vrecall=99.7559, vprecision=99.7683, vcoverage=98.0678, 
Extend: f1score=99.6394, recall=99.5627, precision=99.7161, vf1score=99.8764, vrecall=99.8785, vprecision=99.8744, vcoverage=98.5437, 
Extend: f1score=99.6951, recall=99.6396, precision=99.7507, vf1score=99.8832, vrecall=99.8842, vprecision=99.8822, vcoverage=98.4070, 
Extend: f1score=99.7073, recall=99.6477, precision=99.7669, vf1score=99.8846, vrecall=99.8908, vprecision=99.8784, vcoverage=98.5995, 
Extend: f1score=99.7134, recall=99.6619, precision=99.7649, vf1score=99.8889, vrecall=99.9012, vprecision=99.8766, vcoverage=98.6860, 
Extend: f1score=99.7317, recall=99.6984, precision=99.7650, vf1score=99.9004, vrecall=99.9240, vprecision=99.8768, vcoverage=98.7615, 
Extend: f1score=99.7327, recall=99.7145, precision=99.7509, vf1score=99.8923, vrecall=99.9138, vprecision=99.8708, vcoverage=98.8599, 
Extend: f1score=

KeyboardInterrupt: 

In [123]:
current_result_df

,name,f1score,Truth,Predicted,loss,Proba,Path


In [115]:
models_df.head()

,epoch,path,labels,predicted,proba,vf1score,vloss,tf1score,tloss
18,18,./traces/obfusWS01/model_train_nontrivialjunk_...,"[1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, ...","[1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, ...",[/ws/mnt/local/data/output/obfus/drebinObfus/0...,0.877969,0.008500,0.995934,0.000441
6,6,./traces/obfusWS01/model_train_nontrivialjunk_...,"[1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, ...","[1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, ...",[/ws/mnt/local/data/output/obfus/drebinObfus/0...,0.900546,0.006033,0.993451,0.000735
14,14,./traces/obfusWS01/model_train_nontrivialjunk_...,"[1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, ...","[1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, ...",[/ws/mnt/local/data/output/obfus/drebinObfus/0...,0.922614,0.006308,0.993452,0.000630
5,5,./traces/obfusWS01/model_train_nontrivialjunk_...,"[1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, ...","[0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, ...",[/ws/mnt/local/data/output/obfus/drebinObfus/0...,0.933005,0.002839,0.990639,0.000918
35,35,./traces/obfusWS01/model_train_nontrivialjunk_...,"[1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, ...","[0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, ...",[/ws/mnt/local/data/output/obfus/drebinObfus/0...,0.939498,0.003496,0.996273,0.000389


In [104]:
models_df = df.models.iloc[0]
result_df = df.evalResuls.iloc[0]
models_df.sort_values(by=['vloss', 'tloss'], inplace=True)
selectedModelPaths = models_df.path.iloc[:ensembleSize].tolist()
current_result_df = result_df.loc[result_df.name.isin(selectedModelPaths)]

evalDataset(current_result_df)

Extend: f1score=99.5016, recall=99.4308, precision=99.5725, vf1score=99.8765, vrecall=99.8149, vprecision=99.9382, vcoverage=99.1739, 


In [21]:
selectedModelPaths

['./traces/detectionWS01/model_train_amd_090.pth']

In [7]:
amd_df.evalResuls.iloc[0]

,name,f1score,Truth,Predicted,loss,Proba,Path
0,./traces/detectionWS01/model_train_amd_090.pth,0.993836,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.000300,"[0.9999993, 0.11785824, 3.8572493e-06, 2.73771...",[/ws/mnt/local/data/output/datasets/amd/c08112...
1,./traces/detectionWS01/model_train_amd_094.pth,0.993615,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...","[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.000285,"[0.9999933, 0.8796648, 3.322431e-08, 8.191548e...",[/ws/mnt/local/data/output/datasets/amd/c08112...
2,./traces/detectionWS01/model_train_amd_080.pth,0.993178,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...","[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.000306,"[0.9999995, 0.52806145, 1.1709273e-05, 4.64785...",[/ws/mnt/local/data/output/datasets/amd/c08112...
3,./traces/detectionWS01/model_train_amd_081.pth,0.992000,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.000321,"[0.99999404, 0.019538669, 0.0056791822, 2.6667...",[/ws/mnt/local/data/output/datasets/amd/c08112...
4,./traces/detectionWS01/model_train_amd_069.pth,0.993187,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.000290,"[0.99999905, 0.33358666, 2.4010067e-06, 8.2722...",[/ws/mnt/local/data/output/datasets/amd/c08112...
5,./traces/detectionWS01/model_train_amd_076.pth,0.991991,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...","[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.000332,"[0.99999845, 0.7918521, 0.00033416486, 1.39966...",[/ws/mnt/local/data/output/datasets/amd/c08112...
6,./traces/detectionWS01/model_train_amd_068.pth,0.993614,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...","[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.000315,"[0.99999475, 0.8816969, 1.4293153e-10, 3.59530...",[/ws/mnt/local/data/output/datasets/amd/c08112...
7,./traces/detectionWS01/model_train_amd_042.pth,0.992090,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.000323,"[0.9999863, 0.04765592, 2.700389e-07, 0.000198...",[/ws/mnt/local/data/output/datasets/amd/c08112...
8,./traces/detectionWS01/model_train_amd_064.pth,0.991536,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.000327,"[0.99998057, 0.04596449, 2.8663774e-07, 1.1664...",[/ws/mnt/local/data/output/datasets/amd/c08112...
9,./traces/detectionWS01/model_train_amd_070.pth,0.993735,"[1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...","[1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 1, 1, ...",0.000274,"[0.9996056, 0.024373388, 1.5295343e-05, 5.9939...",[/ws/mnt/local/data/output/datasets/amd/c08112...


In [5]:
amd_df.models.iloc[0]

,epoch,path,labels,predicted,proba,vf1score,vloss,tf1score,tloss
90,90,./traces/detectionWS01/model_train_amd_090.pth,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...",[/ws/mnt/local/data/output/datasets/zoo/b179bf...,0.993562,0.000276,0.996856,0.000221
94,94,./traces/detectionWS01/model_train_amd_094.pth,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...",[/ws/mnt/local/data/output/datasets/zoo/b179bf...,0.994196,0.000277,0.996762,0.000226
80,80,./traces/detectionWS01/model_train_amd_080.pth,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...",[/ws/mnt/local/data/output/datasets/zoo/b179bf...,0.993985,0.000279,0.996701,0.000242
81,81,./traces/detectionWS01/model_train_amd_081.pth,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...",[/ws/mnt/local/data/output/datasets/zoo/b179bf...,0.993169,0.000284,0.996948,0.000221
69,69,./traces/detectionWS01/model_train_amd_069.pth,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...",[/ws/mnt/local/data/output/datasets/zoo/b179bf...,0.993155,0.000286,0.996763,0.000247
76,76,./traces/detectionWS01/model_train_amd_076.pth,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...",[/ws/mnt/local/data/output/datasets/zoo/b179bf...,0.993367,0.000288,0.996763,0.000243
68,68,./traces/detectionWS01/model_train_amd_068.pth,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...",[/ws/mnt/local/data/output/datasets/zoo/b179bf...,0.993776,0.000292,0.995930,0.000293
42,42,./traces/detectionWS01/model_train_amd_042.pth,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...",[/ws/mnt/local/data/output/datasets/zoo/b179bf...,0.993138,0.000296,0.994295,0.000428
64,64,./traces/detectionWS01/model_train_amd_064.pth,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, ...",[/ws/mnt/local/data/output/datasets/zoo/b179bf...,0.993347,0.000297,0.995868,0.000290
70,70,./traces/detectionWS01/model_train_amd_070.pth,"[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...","[0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 0, 1, ...",[/ws/mnt/local/data/output/datasets/zoo/b179bf...,0.994599,0.000299,0.996239,0.000295
